In [30]:
# Import necessary libraries
import openai
from openai import OpenAI
from typing_extensions import override
from openai import AssistantEventHandler, OpenAI

In [31]:
# Create a GPT Assistant

# OpenAI instance
client = OpenAI(api_key='API-KEY')

assistant = client.beta.assistants.create(
    name="Diabetes AI Assistant",
    description="An assistant that walks you through understanding diabetes.",
    instructions = "Tell me more about diabetes.",
    tools=[{"type": "file_search"}],
    model = "gpt-3.5-turbo"
)


In [32]:
# Create a vector store called "Diabetes Data"
vector_store = client.beta.vector_stores.create(name="Diabetes Data")

In [33]:
# Ready the files for upload to OpenAI
file_paths = ["data/diabetes.txt"]
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)



completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


In [34]:
# Update the assistant to use the vector store
client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}}
)

Assistant(id='asst_Oa8VlPuoerDBIoU9TFM0ZpW2', created_at=1716789100, description='An assistant that walks you through understanding diabetes.', instructions='Tell me more about diabetes.', metadata={}, model='gpt-3.5-turbo', name='Diabetes AI Assistant', object='assistant', tools=[FileSearchTool(type='file_search')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=['vs_9V3QYdqjQi2gRXJFbTvCuNJY'])), top_p=1.0)

In [35]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
    file=open("diabetes.txt", "rb"), purpose="assistants"
)
 
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "What is Diabetes mellitus?",
            # Attach the new file to the message.
            "attachments": [
                {"file_id": message_file.id, "tools": [{"type": "file_search"}]}
            ],
        }
    ]
)
 
# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_z0HR92sIIaFgKB92z9zsQhMs'])


In [36]:
# Define the event handler
class EventHandler(AssistantEventHandler):
    def on_text_created(self, text) -> None:
        print(f"\nassistant > {text}", end="", flush=True)

    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    def on_message_done(self, message) -> None:
        # print a citation to the file searched
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))

# Use the stream SDK helper with the EventHandler class to create the Run
with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account.",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()

